In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Multivariate Analysis

### Introduction

In this project, we will experiment and evaluate different methods of optimising a set of analysis cuts. This is used to simplify the dataset in order to then determine if an experiment's outcome is significant enough to call a new discovery. 

We will determine the optimal cut, maximising the significance of the signal events using the following formula: 

\begin{equation}
s = \frac{N_s}{\sqrt{N_b}}
\end{equation}

### Goal 1

Firstly, we will create a class for generating the signal and background toy distributions. 

Using the Monte Carlo rejection sampling method, we generated the events we will then perform the cuts on.

In [ ]:
class Distribution: 
    '''
        A class to create the distributions for the background and signal using the Monte Carlo method.

        Attributes: 
        -----------
            mean_s : int
                mean of the signal distribution
            width_s : int
                width of the signal distribution
            mean_b : int
                mean of the background distribution
            width_b : int
                width of the background distribution
            events_s : int
                number of signal events
            events_b : int
                number of background events

        Methods: 
        --------
        gaussian():
            this creates the distribution using the Monte Carlo method of rejection sampling

    '''

    def __init__(self, mean_s, width_s, mean_b, width_b, events_s, events_b):
        '''
        Construct all the necessary attributes for the Distribution object.

        Parameters: 
        -----------
            mean_s : int
                mean of the signal distribution
            width_s : int
                width of the signal distribution
            mean_b : int
                mean of the background distribution
            width_b : int
                width of the background distribution
            events_s : int
                number of signal events
            events_b : int
                number of background events

        '''
        self.mean_s = mean_s
        self.width_s = width_s
        self.mean_b = mean_b
        self.width_b = width_b
        self.events_s = events_s
        self.events_b = events_b
        

    def gaussian(self):
        '''
        This is where we will create the gaussian distributions.

        Parameters
        ----------
        None

        Returns
        --------
        x_s : NpArray
            array of x coordinates of signal events
        x_b : NpArray
            array of x coordinates of background events
        '''

        x_s = np.array([])
        x_b = np.array([])

        # for 'random sampling' we first created a large number of random numbers to find the maximum value of the distribution function
        a_s = np.random.uniform(-3*self.width_s + self.mean_s, 3*self.width_s + self.mean_s, 1000)
        sigma_s = self.width_s/2
        p_a_s = 1/(sigma_s*np.sqrt(2*np.pi))*np.exp(-1/2*((a_s-self.mean_s)/sigma_s)**2)
        p_max_s = np.max(p_a_s)

        a_b = np.random.uniform(-3*self.width_b + self.mean_b, 3*self.width_b + self.mean_b, 1000)
        sigma_b = self.width_b/2
        p_a_b = 1/(sigma_b*np.sqrt(2*np.pi))*np.exp(-1/2*((a_b-self.mean_b)/sigma_b)**2)
        p_max_b = np.max(p_a_b)

        # here, we implement rejection sampling for creating the gaussian distributions
        while len(x_s) < self.events_s:
            x_trial_s = np.random.uniform(-3*self.width_s + self.mean_s, 3*self.width_s + self.mean_s)
            p_x_s = 1/(sigma_s*np.sqrt(2*np.pi))*np.exp(-1/2*((x_trial_s-self.mean_s)/sigma_s)**2)
            c_s = np.random.uniform(0, p_max_s)
            if p_x_s > c_s:
                x_s = np.append(x_s, x_trial_s) 

        while len(x_b) < self.events_b:
            x_trial_b = np.random.uniform(-3*self.width_b + self.mean_b, 3*self.width_b + self.mean_b)
            p_x_b = 1/(sigma_b*np.sqrt(2*np.pi))*np.exp(-1/2*((x_trial_b-self.mean_b)/sigma_b)**2)
            c_b = np.random.uniform(0, p_max_b)
            if p_x_b > c_b:
                x_b = np.append(x_b, x_trial_b) 

        # ordering the NpArrays
        x_s = np.sort(x_s)
        x_b = np.sort(x_b)

        return x_s, x_b



We will illustrate the correct implementation of the Monte Carlo method by plotting the histograms of the generated signal and background distributions, alongside the corresponding analytical gaussian distributions generated by Python's in-built functions.

In [ ]:
x_s, x_b = Distribution(15, 2, 10, 5, 1000, 2000).gaussian()
bins = np.linspace(0, 25, 50)

plt.hist(x_s, bins = bins, histtype = 'step')
plt.hist(x_b, bins = bins, histtype = 'step')

# analytic signal distribution
x_axis_s = np.arange(0, 25, 0.01)
plt.plot(x_axis_s, norm.pdf(x_axis_s, 15,2 )*1000)

# analytic background sitribution
x_axis_b = np.arange(0, 25, 0.01)
plt.plot(x_axis_b, norm.pdf(x_axis_b, 10, 5)*2000)

plt.show()

### Goal 2

Here we will implement the bisection method to maximise the significance in order to determine the optimal cut. 

Firstly, however, we will calcualte the significance values for every possible cut value and plot the graph.

The optimal cut value result outputted from this function will then be used for testing the validity of the bisection and the Nelder Mead method, both of which will be applied to the list of significance vales.

In [ ]:
def significance(x_s, x_b):
    '''
    In this function we  plot the significance values against the x values and determine the optimal cut.

    Parameters
    ----------
        x_s : NpArray
            array contatining the x values for the signal events.
        x_b : NpArray
            array contatining the x values for the background events.

    Returns
    --------
        sign : list
            list of the signinificance values
        # cut_value : int
            the x value of the maximum significance
        x_axis : list 
            list of values of the x axis 

    '''

    x = np.sort(np.concatenate((x_s, x_b)))
    x_copy = x
    sign = []
    x_axis = []

    while x[0] != x_s[-1]:
        s = len([i for i in x_s if i >= x[0]])/np.sqrt(len([j for j in x_b if j >= x[0]]))
        x_axis.append(x[0])
        x = x[1: ]
        sign.append(s)

    # index for value of the maximum significance
    index = np.where(sign == np.max(sign))[0][0]
    cut_value = x_copy[index]

    return sign, cut_value, x_axis



In [ ]:
# plot the significance values against all possible cut values (list of combined positions of signal and backgound events)
x_s, x_b = Distribution(15, 2, 10, 5, 1000, 2000).gaussian()
sign, cut_value, x_axis = significance(x_s, x_b)

plt.plot(x_axis, sign)

Now, we implement the bisection method, finding the ideal cut by maximising the significance.

We chose two boundaries, a and b, expecting the maximum significance to be between them. Then, we calculate the midpoint,of these two, m, and two futher midpoints, l and r, between the initial boundaries and their centre m. We evaluate the value of the significance at each of the five points and then redefine them accordingly. Repeat this process until either a number of iterations has been reached, or the boundaries, a and b, have a small enough gap between them. 

In [ ]:
def bisection(x_axis, f, lower, upper, gap, iterations):
    '''
    Function implementing the bisection method, in order to find the cut value for maximum significance. 

    Parameters
    ----------
        x_axis : list
            list of all the x values.
        f : list 
            the list of values of function we wish to maximise - in our case this will be the significance.
        lower : int
            lower initally imposed bound
        upper : int
            upper initially imposed bound
        gap : int
            minimum distance between the found boundaries for bisection method to terminate.
        iterations : int
            number of iterations 

    Returns 
    -------
        cut_value : int
            the location on the x axis where the maximum significance was found

    '''
    
    a = [i for i,v in enumerate(x_axis) if v > lower][0]
    b = [i for i,v in enumerate(x_axis) if v > upper][0]

    for _ in range(iterations):

        m = int(a + (b - a)/2)
        l = int(a + (m - a)/2)
        r = int(m + (b - m)/2)

        # indexes 0, 1, 2, 3, 4
        for i in [a, l, m, r, b]:
            values = f[i]

        if f[0] == np.max(f) or f[1] == np.max(f):
            b = m
        
        elif f[3] == np.max(f) or f[4] == np.max(f):
            a = m

        else:
            a = l
            b = r
        
        if (b - a) < gap:
            break

    cut_value = x_axis[a]

    return cut_value

Now, we will compare the cut value obtained from calculating the significances and finding the maximum value, versus applying the bisection method on the significance curve.

NOTE: in this Jupyter Notebook, I believe there is a problem with not being able to call the NpArrays from the significance() function, unless it was the previous one to be run. Therefore, I have repeated the function (and collapsed it s description for it to not bother much) in multiple cells that were meant for illustrating the validity of the methods.

In [ ]:
def significance(x_s, x_b):
    x = np.sort(np.concatenate((x_s, x_b)))
    x_copy = x
    sign = []
    x_axis = []

    while x[0] != x_s[-1]:
        s = len([i for i in x_s if i >= x[0]])/np.sqrt(len([j for j in x_b if j >= x[0]]))
        x_axis.append(x[0])
        x = x[1: ]
        sign.append(s)

    # index for value of the maximum significance
    index = np.where(sign == np.max(sign))[0][0]
    cut_value = x_copy[index]

    return sign, cut_value, x_axis
x_s, x_b = Distribution(15, 2, 10, 5, 1000, 2000).gaussian()
sign, cut_value, x_axis = significance(x_s, x_b)
print('cut value from finding the maximum from the significances list: ',cut_value)

cut_value_bisection = bisection(x_axis, sign, 13, 15, 0.01, 25)
print('cut value found via the bisection method: ', cut_value_bisection)

### Goal 3

We generate 1000 toy experiments, with the number of signal and background events varying according to a poisson distribion. We will apply the cut value found via calculating all significance values and finding the maximum value, and calculate the significance for each of the toy experiements, and a histogram of these significances. 

In [ ]:
def repeat_experiment(cut_value, mean_s, width_s, mean_b, width_b, events_s, events_b, n):
    '''
    We vary the number of signal and backgound events obaying a poisson distribution and repeat the experiment n times, applying a cut value and calculating the significance each time.

    Parameters
    ----------
        cut_value : int
            the optimal cut value found maximising the significance
        mean_s : int
            mean of the signal distribution
        width_s : int
            width of the signal distribution
        mean_b : int
            mean of the background distribution
        width_b : int
            width of the background distribution
        events_s : int
            number of signal events
        events_b : int
            number of background events
        n : int
            number of times the experiment is repeated


    Returns
    -------
        significance : NpArray
            stores the significances of each experiment using the given cut value

    '''

    # generating the poisson distributed number of signal and background events 
    nr_background = np.random.poisson(lam = events_b, size = n)
    nr_signal = np.random.poisson(lam = events_s, size = n)

    significance = np.array([])

    # calculating and storing the significance for each experiment
    for i, j in zip(nr_signal, nr_background):
        x_s, x_b = Distribution(mean_s, width_s, mean_b, width_b, i, j).gaussian()
        s = len([i for i in x_s if i >= cut_value])/np.sqrt(len([j for j in x_b if j >= cut_value]))
        significance = np.append(significance, s)
        

    return significance



In [ ]:
def significance(x_s, x_b):
    x = np.sort(np.concatenate((x_s, x_b)))
    x_copy = x
    sign = []
    x_axis = []

    while x[0] != x_s[-1]:
        s = len([i for i in x_s if i >= x[0]])/np.sqrt(len([j for j in x_b if j >= x[0]]))
        x_axis.append(x[0])
        x = x[1: ]
        sign.append(s)

    # index for value of the maximum significance
    index = np.where(sign == np.max(sign))[0][0]
    cut_value = x_copy[index]

    return sign, cut_value, x_axis
# Plotting the histogram of the significance values applying a fixed cut value. (will take around 1m 42s)
x_s, x_b = Distribution(15, 2, 10, 5, 1000, 2000).gaussian()
sign, cut_value , x_axis = significance(x_s, x_b)

cut_value = bisection(x_axis, sign, 13, 15, 0.001, 25)
significance = repeat_experiment(cut_value, 15, 2, 10, 5, 1000, 2000, 1000)

plt.hist(significance, bins = 30)

### Goal 4 Nelson Mead Method

In this section, we will implement the Nelder Mead method in one dimension. This method is used to find the minimum or maximum of a given function. In our case, I have chosed to flip the significance curve by just applying a negative sign to every value in the list, in order work with minimising the function, rather than maximising it.

Firstly, I created a helper function which helps find the closeset existing x value to one that we are looking for. As signal and background events are created randomly, there is no certainty that a value (for example a midpoint) that we are looking for, has a corresponding event at that location.

In [ ]:
def sig_for_x(x, x_axis, sign):
    '''
    This function helps us find the nearest x value (and hence significance) to the searched value. 

    Parameters 
    ----------
        x : int
            value we are looking for
        x_axis : list
            list of x values generated
        sign : list 
            list of the corresponding significance values

    Returns
    --------
        value_sign : int
            significance value that corresponds to the closest generated x value to the one we were looking for

    '''
    
    if x_axis[-1] < x:
        a = -1
    else:
        a = [i for i,v in enumerate(x_axis) if v >= x][0]

    value_sign = -sign[a]

    return value_sign

Now, we will implement the Nelder Mead method. 

Firstly, order the values of the function (significances) and pick the best, worst and second worst values and the corresponding coordinates. Find the centroid of the coordinates and the reflected point and evaluate the significances for each of them. Using this to guide us, we will redefine the points and, where needed, also calculate the extension points and the contracted points, evaluating the significances and redefining them, discarding the rest of the x coordinates.

Repeat this process until the minimum significance is found and return the corresponding x coordinate.

In [ ]:
def nelson_mead_1d(x_axis, f):
    '''
    This function is applying the Nelder Mead method to minimise a one dimensional function.

    Parameters
    ----------
        x_axis : NpArray
            contains all coordinates of the signal and background events
        f : NpArray
            the coresponding values of the function (in our case significances) for the given x coordinate values

    Returns
    -------
        minimum : int
            the coordinate on the x axis where the minimum is found

    '''

    # inverting the curve to apply minimization
    f = [-i for i in f]
    list = np.column_stack((x_axis, f))
    sort = sorted(list, key = lambda l:l[1])
    s = [l.tolist() for l in sort]
    new_x_list = [row[0] for row in s]
    
    # choosing the best, second to worst and worst points
    x1 = new_x_list[0]
    xn1 = new_x_list[-1]
    xn = new_x_list[-2]

    for _ in range(5):

        # centroid
        x_0 = (sum(new_x_list) - xn1)/(len(new_x_list) - 1)

        # reflection
        x_r = x_0 + 1 * (x_0 - xn1)

        if sig_for_x(x1, x_axis, f) <= sig_for_x(x_r, x_axis, f) and sig_for_x(x_r, x_axis, f) < sig_for_x(xn, x_axis, f):
            xn1 = x_r

        elif sig_for_x(x_r, x_axis, f) < sig_for_x(x1, x_axis, f):
            # expanded point
            x_e = x_0 + 2 * (x_r - x_0)

            if sig_for_x(x_e, x_axis, f) < sig_for_x(x_r, x_axis, f):
                xn1 = x_e

            else: 
                xn1 = x_r

        else: 
            if sig_for_x(x_r, x_axis, f) < sig_for_x(xn1, x_axis, f):
                # contracted point on the outside
                x_c = x_0 + 0.5 * (x_r - x_0)

                if sig_for_x(x_c, x_axis, f) < sig_for_x(x_r, x_axis, f):
                    xn1 = x_c

                else: 
                    # shrink
                    new_x_list = [(x1 + 0.5 * (i - x1)) for i in new_x_list[1:]]
                    new_x_list.insert(0, x1)

            elif sig_for_x(x_r, x_axis, f) >= sig_for_x(xn1, x_axis, f):
                # contracted point on the inside
                x_c = x_0 + 0.5 * (xn1 - x_0)

                if sig_for_x(x_c, x_axis, f) < sig_for_x(xn1, x_axis, f):
                    xn1 = x_c
                
                else:
                    # shrink
                    new_x_list = [(x1 + 0.5 * (i - x1)) for i in new_x_list[1:]]
                    new_x_list.insert(0, x1)

        if np.sort(new_x_list)[-1] < xn1:
            a = np.sort(new_x_list)[-1]

        else:
            a = [v for i,v in enumerate(np.sort(new_x_list)) if v >= xn1][0]

        new_x_list = new_x_list[:new_x_list.index(a)]
        new_x_list.insert(-1, a)
        

        if len(new_x_list) == 1:
            break

        
    minimum = new_x_list[0]

    return minimum


In [ ]:
def significance(x_s, x_b):
    x = np.sort(np.concatenate((x_s, x_b)))
    x_copy = x
    sign = []
    x_axis = []

    while x[0] != x_s[-1]:
        s = len([i for i in x_s if i >= x[0]])/np.sqrt(len([j for j in x_b if j >= x[0]]))
        x_axis.append(x[0])
        x = x[1: ]
        sign.append(s)

    # index for value of the maximum significance
    index = np.where(sign == np.max(sign))[0][0]
    cut_value = x_copy[index]

    return sign, cut_value, x_axis
# proving the efficiency of the method, here we are printing the cut value found via maximum significance from the list versus the Nelder Mead method, and they are identical.
x_s, x_b = Distribution(15, 2, 11, 6, 1000, 2000).gaussian()
sign, cut_value, x_axis = significance(x_s, x_b)
print('via the significance graph: ', cut_value)
print('via the nelder mead: ', nelson_mead_1d(x_axis, sign))

plt.plot(x_axis, sign)

### Goal 5 Lower and Upper Cut 

I found the cuts using histograms and calculating the significance for each bin and the sum-significance for every possible lower and upper cut combination. I stored these in a matrix of three columns [lower_cut, upper_cut, significance], and was then looking for the maximum significance and selecting that row as the answer for optimal cuts.

In [ ]:
def upper_lower(mean_s, width_s, x_s, x_b):
        '''
        In this function, we will find the optimal upper and lower cut on the data, using histograms.

        Parameters
        ----------
                mean_s : int
                        mean of the signal distribution
                width_s : int
                        width of the signal distribution
                x_s : NpArray
                        array containing the x coordinates of the signal events
                x_b : NpArray
                        array containing the x coordinates of the background events

        Returns
        -------
                x_low : int
                        the optimal low cut found
                x_high : int 
                        the optimal high cut found

        '''

        nr_bins = 50
        lower_bound, upper_bound = mean_s - 2.5 * width_s, mean_s + 2.5 * width_s
        bins = np.linspace(lower_bound, upper_bound, nr_bins)

        bac = plt.hist(x_b, bins, histtype = 'step')
        si = plt.hist(x_s, bins, histtype = 'step')

        s = [l.tolist() for l in si[0]]
        b = [l.tolist() for l in bac[0]]

        i_start = [i for i, v in enumerate(s) if v > 0][0]
        s.reverse()
        i_end = [i for i, v in enumerate(s) if v > 0][0]
        s.reverse()

        s = s[i_start:-i_end]
        b = b[i_start:-i_end]

        store = np.array([0, 0, 0])

        for low in range(len(s)):
                for h in range(len(s)-low):
                        high = h + low + 1
                        sign = np.sum(s[low:high])/np.sqrt(np.sum(b[low:high]))
                        add = np.array([low, high, sign])    
                        store = np.vstack((store, add))

        m_s = np.amax(store, axis = -1)

        index = np.where(m_s == np.max(m_s))[0][0]
        optimal = store[index]

        steps_width = (upper_bound - lower_bound)/ nr_bins

        x_low = (steps_width*(optimal[0]*2 + i_start*2 + 1))/2 + (mean_s - 2.5 * width_s)
        x_high = (steps_width*(optimal[1]*2 + i_start*2 + 1))/2 + (mean_s - 2.5 * width_s)


        return x_low, x_high

In [ ]:
def significance(x_s, x_b):
    x = np.sort(np.concatenate((x_s, x_b)))
    x_copy = x
    sign = []
    x_axis = []

    while x[0] != x_s[-1]:
        s = len([i for i in x_s if i >= x[0]])/np.sqrt(len([j for j in x_b if j >= x[0]]))
        x_axis.append(x[0])
        x = x[1: ]
        sign.append(s)

    # index for value of the maximum significance
    index = np.where(sign == np.max(sign))[0][0]
    cut_value = x_copy[index]

    return sign, cut_value, x_axis
# plotting the histogram of signal and backgound events and the found optimal lower and upper cut.
mean_s, width_s = 15, 2
x_s, x_b = Distribution(15, 2, 12, 5, 1000, 2000).gaussian()
upper_lower(15, 2, x_s, x_b)

Generating 1000 toy experiments, varying the number of signal ad background events corresponding a poisson distribution and plotting the histogram of siginificances, given a lower and upper cut.

In [ ]:
def repeat_experiment_up_low(cut_low, cut_high, mean_s, width_s, mean_b, width_b, events_s, events_b, n):
    '''
    We vary the number of signal and backgound events obaying a poisson distribution and repeat the experiment n times, applying a cut value and calculating the significance each time.

    Parameters
    ----------
        cut_low : int
            the optimal lower cut value found maximising the significance
        cut_high : int
            the optimal high cut value found maximising the significance
        mean_s : int
            mean of the signal distribution
        width_s : int
            width of the signal distribution
        mean_b : int
            mean of the background distribution
        width_b : int
            width of the background distribution
        events_s : int
            number of signal events
        events_b : int
            number of background events
        n : int
            number of times the experiment is repeated


    Returns
    -------
        significance : NpArray
            stores the significances of each experiment using the given cut value

    '''

    # generating the poisson distributed number of signal and background events 
    nr_background = np.random.poisson(lam = events_b, size = n)
    nr_signal = np.random.poisson(lam = events_s, size = n)

    significance = np.array([])

    # calculating and storing the significance for each experiment
    for i, j in zip(nr_signal, nr_background):
        x_s, x_b = Distribution(mean_s, width_s, mean_b, width_b, i, j).gaussian()
        s = len([i for i in x_s if (i >= cut_low and i <= cut_high)])/np.sqrt(len([j for j in x_b if (j >= cut_low and j <= cut_high)]))
        significance = np.append(significance, s)

    return significance

In [ ]:
def significance(x_s, x_b):
    x = np.sort(np.concatenate((x_s, x_b)))
    x_copy = x
    sign = []
    x_axis = []

    while x[0] != x_s[-1]:
        s = len([i for i in x_s if i >= x[0]])/np.sqrt(len([j for j in x_b if j >= x[0]]))
        x_axis.append(x[0])
        x = x[1: ]
        sign.append(s)

    # index for value of the maximum significance
    index = np.where(sign == np.max(sign))[0][0]
    cut_value = x_copy[index]

    return sign, cut_value, x_axis
# I have adjusted the mean of the backgound distribution to 12 here, to make sure in each bin where there are signal events, 
# there will also be backgound events, so that the significance is not infinity.
x_s, x_b = Distribution(15, 2, 12, 5, 1000, 2000).gaussian()
sign, cut_value , x_axis = significance(x_s, x_b)

low_cut, high_cut = upper_lower(15, 2, x_s, x_b)
print(low_cut, high_cut)
significance = repeat_experiment_up_low(low_cut, high_cut, 15, 2, 12, 5, 1000, 2000, 1000)

In [ ]:
# Plotting the histogram of the significance values applying the fixed cut values. 
plt.hist(significance, bins = 30)

### Goal 6 

Firstly, we created a more complicated toy distribution with more dimensions.

The 2D distribution has a corresponding 4d graph of significances. for nelder mead - every coordinate will have a value. \
Have a function outside the Nelder mead, calculating the value at that point for lower, upper cut for x and y coordinates. 

Implement the Nelder Mead for multi dimensional graphs and find the optimal point coordinates. 

In [ ]:
# generating the signal and backgound events in 2 dimensions
sigx, sigy, rho = 5, 5, -0.4
cov_mat = [[sigx**2, rho * sigx * sigy], [rho * sigx * sigy, sigy**2]]
sX, sY = np.random.multivariate_normal([8, 10], cov_mat, size=1000).T

b_sigx, b_sigy, b_rho = 10, 15, 0.1
b_cov_mat = [[b_sigx**2, b_rho * b_sigx * b_sigy], [b_rho * b_sigx * b_sigy, b_sigy**2]]
bX, bY = np.random.multivariate_normal([10, 15], b_cov_mat, size=10000).T

plt.scatter(bX, bY)
plt.scatter(sX, sY)

We need a helper function, calculating the significance of a point - counting the number of events from a square of fixed size at some given coordinates. 

In [ ]:
def point(p, sX, sY, bX, bY):
    '''
    Function to find the value of significance for given x, y lower, upper cuts. 

    Parameters
    ---------
        p : NpArray
            gives the coordinates of the point we want the significance from
        sX : NpArray
            array of the x coordinates of the signal events
        sY : NpArray
            array of the y coordinates of the signal events
        bX : NpArray
            array of the x coordinates of the background events
        bY : NpArray
            array of the y coordinates of the background events

    Return
    -------
        significance : int
            value of the significance (Ns/sqrt(Nb)) for given parameters counting the amount of signal and background events in a given box
    
    '''

    nr_s = 0
    nr_b = 0

    size = 5

    for i in range(len(sX)):
        if (sX[i] > p[0] - size) & (sX[i] < p[0] + size) & (sY[i] > p[1] - size) & (sY[i] < p[1] + size) == 1:
            nr_s = nr_s + 1

        if (bX[i] > p[0] - size) & (bX[i] < p[0] + size) & (bY[i] > p[1] - size) & (bY[i] < p[1] + size) == 1:
            nr_b = nr_b + 1

    significance = -(nr_s / np.sqrt(nr_b))

    return significance
        

Now, we will implement the Nelder Mead method on a 2 dimensional space, which means that we will operate with a 3 dimensional simplex. Initially, we will start form 3 random points and odering them into the best, second worst and worst point, calculating their significances. Then, we will calculate their centroid and reflected point, evaluating the significances at every instance and when necessary also calculating the extended and contracted points. After each iteration and evaluation, the simplex points are updated. The process ends either after a certain number of iterations, or when the shortest side of the simplex is under a certain given value. The output of the Nelder Mead method is the point of maximum significance.

In [ ]:
def nelder_mead_2d(sX, sY, bX, bY, times, gap):
    '''
    Implementing the multi dimensional Nelder Mead simplex method on the generated scattered two dimensional signal and backgound events. 

    Parameters 
    ----------
        sX : NpArray
            array of the x coordinates of the signal events
        sY : NpArray
            array of the y coordinates of the signal events
        bX : NpArray
            array of the x coordinates of the background events
        bY : NpArray
            array of the y coordinates of the background events
        times : int
            number of times for the method to run before selecting the best point - termination condition
        gap : int
            size of the shortest distance between the points in the simplex - termination condition



    Returns
    -------
        p1 : NpArray
            coordinates for the point where the significance is maximised
        sign : int
            maximum significance found

    '''

    # generating 3 random points for the starting Nelder Mead simplex
    p1 = [sX[0], sY[0]]
    pn = [sX[30], sY[5]]
    pn1 = [sX[10], sY[30]]

    for i in range(times):

        p_list = [p1, pn, pn1]
    
        # ordering the significances 
        s = [point(p1, sX, sY, bX, bY), point(pn, sX, sY, bX, bY), point(pn1, sX, sY, bX, bY)]
        s_ord = np.sort(s)
        p1 = p_list[np.where(s == s_ord[0])[0][0]]
        pn = p_list[np.where(s == s_ord[1])[0][0]]
        pn1 = p_list[np.where(s == s_ord[-1])[0][0]]

        # centroid point
        p0 = [(p1[0] + pn[0])/2, (p1[1] + pn[1])/2]

        # reflected point
        pr = [p0[0] + 1 * (p0[0] - pn1[0]), p0[1] + 1 * (p0[1] - pn1[1])]
        

        if point(p1, sX, sY, bX, bY) <= point(pr, sX, sY, bX, bY) and point(pr, sX, sY, bX, bY) < point(pn, sX, sY, bX, bY):
            pn1 = pr

        elif point(pr, sX, sY, bX, bY) < point(p1, sX, sY, bX, bY):
            # expanded point
            pe = [p0[0] + 2 * (pr[0] - p0[0]), p0[1] + 2 * (pr[1] - p0[1])]

            if point(pe, sX, sY, bX, bY) < point(pr, sX, sY, bX, bY):
                pn1 = pe

            else: 
                pn1 = pr
        
        else: 
            if point(pr, sX, sY, bX, bY) < point(pn1, sX, sY, bX, bY):
                # contracted point on outside
                pc = [p0[0] + 0.5 * (pr[0] - p0[0]), p0[1] + 0.5 * (pr[1] - p0[1])]

                if point(pc, sX, sY, bX, bY) < point(pr, sX, sY, bX, bY):
                    pn1 = pc

                else: 
                    # shrink
                    pn = [pn[0] + 0.5 * (pn[0] - p1[0]), pn[1] + 0.5 * (pn[1] - p1[1])]
                    pn1 = [pn1[0] + 0.5 * (pn1[0] - p1[0]), pn1[1] + 0.5 * (pn1[1] - p1[1])]



            elif point(pr, sX, sY, bX, bY) >= point(pn1, sX, sY, bX, bY):
                # contracted point on the inside
                pc = [p0[0] + 0.5 * (pn1[0] - p0[0]), p0[1] + 0.5 * (pn1[1] - p0[1])]

                if point(pc, sX, sY, bX, bY) < point(pn1,  sX, sY, bX, bY):
                    pn1 = pc
                
                else:
                    # shrink
                    pn = [pn[0] + 0.5 * (pn[0] - p1[0]), pn[1] + 0.5 * (pn[1] - p1[1])]
                    pn1 = [pn1[0] + 0.5 * (pn1[0] - p1[0]), pn1[1] + 0.5 * (pn1[1] - p1[1])]


            
            # termination condition: checking if the shortest side of the simplex is smaller than the selected value of the gap
            dist = [np.sqrt((pn1[0] - pn[0])**2 + (pn1[1] - pn[1])**2), np.sqrt((pn1[0] - p1[0])**2 + (pn1[1] - p1[1])**2), np.sqrt((p1[0] - pn[0])**2 + (p1[1] - pn[1])**2)]

            if np.max(dist) < gap: 
                break

    sign = -point(p1, sX, sY, bX, bY)
            
    return p1, sign
            
            

In [ ]:
sX, sY = np.random.multivariate_normal([1, 5], cov_mat, size=1000).T
bX, bY = np.random.multivariate_normal([10, 15], b_cov_mat, size=10000).T

p1, sign = nelder_mead_2d(sX, sY, bX, bY, 20, 1)
print('the ideal point for the cut found has the coordinates: ', p1)
print('the significance at this point is: ', sign)

### Conclusion

In conclusion, in order to find the optimal cut value on a dataset and maximise the significance various methods can be used, such as the bisection method or the Nelder Mead method, both of which have been explored in this project. To further optimise the data analysis, both low and high cut values can be found (Goal 5), and the Nelder Mead method can be applied for a more complicated model where more variables should be incorporated.